In [1]:
import tensorflow as tf
import os
import sys
from src import preproc, feed, model_fn
from tqdm import tqdm_notebook
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

%load_ext autoreload
%autoreload 2

In [2]:
data = preproc.read_data(db_path='data/jokes.db', duplicates=False)
data_len = len(data)
data = data[:data_len//3]
vocab = preproc.Vocab(data)
char2idx, idx2char = vocab.build_vocab()

219162it [00:01, 127410.33it/s]


In [3]:
CONFIG_DIR = 'config/'
LOG_DIR = 'logs/'

params = {
    'vocab': char2idx,
    
    "encoder_lstm_size": 200,
    "encoder_num_layers": 2,
    "encoder_final_context_dims": 200,
    "encoder_vocab_size": len(char2idx),
    "encoder_state_size": 60,
    "encoder_emb_size": 100,
    "encoder_vocab": char2idx,
    "initial_state_zero": True,

    "decoder_lstm_size": 200,
    "decoder_vocab_size": len(char2idx),
    "decoder_max_len": 300,
    "decoder_vocab": char2idx,
    "reverse_decoder_vocab": idx2char,
    "decoder_emb_size": 100,

    "use_encoder_embeddings": True,
    "learning_rate": 1e-3,
    "epochs": 100,
    "batch_size": 128,
    "clip": .5
}

with open(os.path.join(CONFIG_DIR, 'params.json'), 'w') as f:
    json.dump(params, f, indent=4, sort_keys=True)

In [4]:
docs_ids = [vocab.tokenize(text) for text in tqdm_notebook(data)]

docs_train, docs_val = train_test_split(
    docs_ids, test_size=0.10, random_state=42)
docs_val, docs_dev = train_test_split(
    docs_val, test_size=0.5, random_state=42)

print('train size:', len(docs_train), len(docs_train)/len(docs_ids))
print('val size:', len(docs_val), len(docs_val)/len(docs_ids))
print('dev size:', len(docs_dev), len(docs_dev)/len(docs_ids))


train size: 65748 0.8999917868973636
val size: 3653 0.050004106551318206
dev size: 3653 0.050004106551318206


In [5]:
docs_train = shuffle(docs_train, random_state=44)

In [23]:
feed_ = feed.Feed()
train_input_fn = feed_.input_fn(docs_train, docs_train, params)
eval_input_fn = feed_.input_fn(docs_val, docs_val, params)
predict_input_fn = feed_.predict_input_fn(params=params, n=10)

In [24]:
cfg = tf.estimator.RunConfig(
    keep_checkpoint_max=150,
    save_checkpoints_secs=60*60,
)
estimator = tf.estimator.Estimator(model_fn.model_fn, os.path.join(LOG_DIR, 'model1/'), cfg, params)

INFO:tensorflow:Using config: {'_model_dir': 'logs/model1/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 3600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 150, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77554b3080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
predictions = []

for p in estimator.predict(input_fn=predict_input_fn):
    predictions.append(p)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize cell states as zeros
INFO:tensorflow:<tf.Variable 'decoder_embeddings:0' shape=(106, 100) dtype=float32_ref>
INFO:tensorflow:Tensor("Fill:0", shape=(?,), dtype=int32)
INFO:tensorflow:Tensor("ToInt32_1/x:0", shape=(), dtype=int32)
INFO:tensorflow:Tensor("dense/BiasAdd:0", shape=(?, ?, 106), dtype=float32)


ValueError: The two structures don't have the same nested structure.

First structure: type=Tensor str=Tensor("dense/BiasAdd:0", shape=(?, ?, 106), dtype=float32)

Second structure: type=tuple str=(200, 200)

More specifically: Substructure "type=tuple str=(200, 200)" is a sequence, while substructure "type=Tensor str=Tensor("dense/BiasAdd:0", shape=(?, ?, 106), dtype=float32)" is not
Entire first structure:
.
Entire second structure:
(., .)